#Downloading data and necessary imports


In [ ]:
!gdown 1MuDMPbzadGkWgef66b1rSZsOnhfoWMcA
!gdown 12jfcFRM7GQrTvrFfS75TJw2mIT0QQF8P
!gdown 1_t8fRolG-tYVnDAfo4WASvL7k2UwNh6G
!gdown 1x5AzGBePqEL5CZrT-04I_BbPqsmZIixc
!gdown 1isKU6GHFzuxogVoSrSW1yr5md7M-17tD
!gdown 1DqBzFECvEjcmKouuCXo8KR70ZHDaZfey

Downloading...
From: https://drive.google.com/uc?id=1MuDMPbzadGkWgef66b1rSZsOnhfoWMcA
To: /content/test.modern.nltktok
100% 66.7k/66.7k [00:00<00:00, 101MB/s]
Downloading...
From: https://drive.google.com/uc?id=12jfcFRM7GQrTvrFfS75TJw2mIT0QQF8P
To: /content/test.original.nltktok
100% 70.8k/70.8k [00:00<00:00, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_t8fRolG-tYVnDAfo4WASvL7k2UwNh6G
To: /content/train.modern.nltktok
100% 919k/919k [00:00<00:00, 131MB/s]
Downloading...
From: https://drive.google.com/uc?id=1x5AzGBePqEL5CZrT-04I_BbPqsmZIixc
To: /content/train.original.nltktok
100% 980k/980k [00:00<00:00, 174MB/s]
Downloading...
From: https://drive.google.com/uc?id=1isKU6GHFzuxogVoSrSW1yr5md7M-17tD
To: /content/valid.modern.nltktok
100% 55.3k/55.3k [00:00<00:00, 87.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DqBzFECvEjcmKouuCXo8KR70ZHDaZfey
To: /content/valid.original.nltktok
100% 58.1k/58.1k [00:00<00:00, 116MB/s]


In [ ]:
!python -m spacy download en_core_web_sm

2023-11-29 15:28:44.385580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 15:28:44.385644: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 15:28:44.385684: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 15:28:44.394582: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 15:28:45.839701: W tensorflow/c

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
import torch
from torchtext.data.metrics import bleu_score
import math
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import time
from tqdm import tqdm
import spacy
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset
import random as random
from torch.nn.utils.rnn import pad_sequence
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW
from sklearn.model_selection import train_test_split


We want to run it on a GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Function for reading the data

In [ ]:
def path_to_list(file):
    f = open(file, "r")
    text = f.read()

    li = str.splitlines(text)
    return li

#Preprocessing the data

Reading the training, test and evaluation data

In [ ]:
original_li = path_to_list("/content/train.original.nltktok")
modern_li = path_to_list("/content/train.modern.nltktok")
original_val = path_to_list("/content/valid.original.nltktok")
modern_val = path_to_list("/content/valid.modern.nltktok")
original_test = path_to_list("/content/test.original.nltktok")
modern_test = path_to_list("/content/test.modern.nltktok")

In [ ]:
tr=len(original_li)
va=len(original_val)
te=len(original_test)
s = len(original_li)+len(original_val)+len(original_test)

In [ ]:
te/s*100

6.937129300118625

In [ ]:
va/s*100

5.779359430604982

In [ ]:
tr/s*100

87.28351126927639

In [ ]:
o_li= original_li + original_val + original_test
m_li = modern_li + modern_val+ modern_test

In [ ]:
modern_li, modern_temp, original_li, original_temp = train_test_split(m_li, o_li, test_size=0.2, random_state=42)

modern_val, modern_test, original_val, original_test = train_test_split(modern_temp, original_temp, test_size=0.5, random_state=42)

In [ ]:
tr=len(original_li)
va=len(original_val)
te=len(original_test)
s = len(original_li)+len(original_val)+len(original_test)

In [ ]:
te/s*100

10.002372479240806

In [ ]:
va/s*100

9.997627520759194

In [ ]:
tr/s*100

80.0

Custom function for word tokenization
- we should change something here, instead of using this word tokenization we could use pretrained word tokenization models
- Spacy for english to tokenize both Shakepserean english and modern english

In [ ]:
def avg_length(li):
    s = 0
    c = 0
    for sentence in li:
        for word in sentence.split():
            s += 1

        c += 1
    return int(s // c)

In [ ]:
src_avg = avg_length(original_li)
trg_avg = avg_length(modern_li)
print(src_avg)
print(trg_avg)

11
10


In [ ]:
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize(text):
    return ["<sos>"]+ [tok.text for tok in spacy_en.tokenizer(text.lower())] + ["<eos>"]

In [ ]:
tokenize(original_li[19])

['<sos>',
 'pluck',
 'up',
 ',',
 'my',
 'heart',
 ',',
 'and',
 'be',
 'sad',
 '.',
 '<eos>']

In [ ]:
src_vocab = build_vocab_from_iterator(map(tokenize, original_li), specials=['<pad>', '<unk>'])
trg_vocab = build_vocab_from_iterator(map(tokenize, modern_li), specials=['<pad>', '<unk>'])

In [ ]:
PAD_IDX = src_vocab.get_stoi()["<pad>"]
print(PAD_IDX)
UNK_IDX = src_vocab.get_stoi()["<unk>"]
print(UNK_IDX)

0
1


In [ ]:
print("Vocabulary:", src_vocab.get_itos())
print("Numericalized Vocabulary:", src_vocab.get_stoi())

Vocabulary: ['<pad>', '<unk>', '<eos>', '<sos>', ',', '.', 'i', 'the', 'and', 'to', 'you', '?', 'of', 'my', 'a', 'that', 'is', 'in', 'not', 'me', 'it', '!', 'he', 'for', "'s", 'be', 'your', 'with', 'this', 'have', 'but', 'thou', 'will', 'what', 'his', 'him', 'so', 'as', "'", 'do', 'no', 'her', 'we', 'are', 'if', 'all', 'shall', 'thee', 'good', 'thy', 'lord', 'by', 'sir', 'now', 'come', '"', 'she', 'well', 'on', 'am', 'our', 'let', 'here', "'ll", 'there', 'they', 'would', 'how', 'then', 'love', 'from', 'at', 'man', 'more', 'go', 'was', 'or', 'o', 'know', '-', 'did', 'can', 'why', 'when', 'them', 'which', 'upon', 'say', 'us', 'hath', 'than', 'their', 'should', 'an', 'one', 'make', 'tis', 'yet', 'must', 'may', 'were', 'where', 'give', 'out', 'some', 'see', 'who', 'like', 'take', ';', 'had', 'speak', 'such', 't', 'tell', 'most', 'think', 'these', 'too', 'mine', 'oh', 'never', 'caesar', 'god', 'hear', 'king', 'father', 'ay', 'much', 'time', 'heart', 'up', 'art', 'th', 'look', 'very', 'nor',

##Creating the dataset and from that creating teh dataloader

In [ ]:
from logging import DEBUG
class ShakespeareanDataset(Dataset):
    def __init__(self, original_sentences, modern_sentences, debug = False):
        self.original_sentences = original_sentences
        self.modern_sentences = modern_sentences
        self.debug = DEBUG

    def __len__(self):
        return len(self.original_sentences)

    def pad_sequence_to_avg_length(self, sequence, avg_length):
        current_length = len(sequence)
        if current_length < avg_length:
            padding = [PAD_IDX] * (avg_length - current_length)
            sequence = sequence + padding
        elif current_length > avg_length:
            sequence = sequence[:avg_length]
        return sequence


    def __getitem__(self, idx):
        src = self.original_sentences[idx]
        trg = self.modern_sentences[idx]

        src_tokens = tokenize(src)
        trg_tokens = tokenize(trg)

        src_indexes = [src_vocab.get_stoi().get(src_tokens[i], UNK_IDX) for i in range(len(src_tokens))]
        trg_indexes = [trg_vocab.get_stoi().get(trg_tokens[i], UNK_IDX) for i in range(len(trg_tokens))]

        src_indexes = self.pad_sequence_to_avg_length(src_indexes,src_avg)
        trg_indexes = self.pad_sequence_to_avg_length(trg_indexes,src_avg)

        if self.debug:
            print(f"Src sentence: {src}")
            print(f"Src tokens: {src_tokens}")
            print(f"Padded Src token indexes: {src_indexes}")

        self.debug = False

        return {
            "src": torch.tensor(src_indexes).to(device),
            "trg": torch.tensor(trg_indexes).to(device)
        }

##Collate function for padding

In [ ]:
# def collate_fn(batch):
#     src_seqs = [item['src'] for item in batch]
#     trg_seqs = [item['trg'] for item in batch]

#     # Find the maximum length in the batch
#     max_src_len = max(len(seq) for seq in src_seqs)
#     max_trg_len = max(len(seq) for seq in trg_seqs)
#     max_len = max(max_src_len, max_trg_len)

#     # Pad sequences to the maximum length
#     src_padded = pad_sequence([torch.nn.functional.pad(seq, pad=(0, max_len - len(seq))) for seq in src_seqs], padding_value=PAD_IDX)
#     trg_padded = pad_sequence([torch.nn.functional.pad(seq, pad=(0, max_len - len(seq))) for seq in trg_seqs], padding_value=PAD_IDX)

#     print()
#     print("-------------------")
#     print("Collate")
#     print("-------------------")
#     print(src_padded.shape)
#     print(trg_padded.shape)
#     return {'src': src_padded, 'trg': trg_padded}

In [ ]:
dataset = ShakespeareanDataset(original_li, modern_li)
dataset_val = ShakespeareanDataset(original_val, modern_val)
dataset_test = ShakespeareanDataset(original_test, modern_test)

batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
data_val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
data_test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [ ]:
print(len(data_loader))

527


#Encoder-Decoder architecture

##<font color="green">Encoder model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout, debug = False):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.dropout = nn.Dropout(dropout)

        self.debug = debug

    def forward(self, src):

        #src = [src len, batch size]

        embedded = self.dropout(self.embedding(src))

        #embedded = [src len, batch size, emb dim]

        outputs, (hidden, cell) = self.rnn(embedded)

        if self.debug:
          print()
          print("-----------------")
          print("Enocder")
          print("-----------------")
          print("->Embedded")
          print(embedded.shape)
          print("->Outputs")
          print(outputs.shape)

        self.debug = False

        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #outputs are always from the top hidden layer

        return hidden, cell

##<font color="red">Decoder model

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout, debug=False):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

        self.debug = debug

    def forward(self, input, hidden, cell):
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]


        #input = [1, batch size]
        if self.debug:
          print()
          print("-----------------")
          print("Decoder")
          print("-----------------")
          print("->Input")
          print(input.shape)


        embedded = self.dropout(self.embedding(input))

        if self.debug:
          print("->Embedded")
          print(embedded.shape)


        #embedded = [1, batch size, emb dim]


        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))

        if self.debug:
          print("->Hidden")
          print(hidden.shape)
          print("->Prediction")
          print(prediction.shape)

        self.debug = False


        #prediction = [batch size, output dim]

        return prediction, hidden, cell

##<font color="purple">Seq2Seq model

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, debug=False):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.debug = debug

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim


        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)


        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)

        if self.debug:
          print()
          print("-----------------")
          print("Seq2Seq")
          print("-----------------")
          print("->src seq2seq")
          print(src.shape)
          print("->trg seq2seq")
          print(trg.shape)
          print("->Seq2Sew hidden:")
          print(hidden.shape)


        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        if self.debug:
          print("->input to seq2seq")
          print(input.shape)


        for t in range(1, trg_len):
            input = input.unsqueeze(0)
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        self.debug = False
        return outputs

#Training and evaluating our model

##Init weights

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

##Train function

In [ ]:
def train(model, data_loader, optimizer, criterion, clip, debug = False):
    model.train()

    epoch_loss = 0
    if debug:
          print()
          print("-----------------")
          print("Training")
          print("-----------------")
          print("->num of 32 batches")
          print(len(data_loader))


    for i, batch in enumerate(tqdm(data_loader, desc="Processing Batches", total=len(data_loader))):
        src = batch["src"]
        trg = batch["trg"]

        src = src.t()
        trg = trg.t()

        if debug:
          print()
          print("-----------------")
          print("Training")
          print("-----------------")
          print("->batch type")
          print(type(batch))

        optimizer.zero_grad()

        output = model(src, trg)

        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]

        output = output[1:].reshape(-1, output_dim)

        # Reshape the target tensor to [(trg len - 1) * batch size]
        trg = trg[1:].reshape(-1)


        # trg = [(trg len - 1) * batch size]
        # output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

        debug = False

    return epoch_loss / len(data_loader)

##Evaluate function

In [ ]:
def evaluate(model, data_loader, criterion, debug=False):
    model.eval()

    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(tqdm(data_loader, desc="Processing Batches", total=len(data_loader))):
            src = batch["src"]
            trg = batch["trg"]
            src = src.t()
            trg = trg.t()
            if debug:
                print()
                print("-----------------")
                print("Evaluation")
                print("-----------------")
                print("->src sha[e]")
                print(src.shape)
                print("->trg sha[e]")
                print(trg.shape)

            output = model(src, trg, 0)  # turn off teacher forcing

            # trg = [trg len, batch size]
            # output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].reshape(-1, output_dim)
            trg = trg[1:].reshape(-1)

            # trg = [(trg len - 1) * batch size]
            # output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()

            debug = False

    return epoch_loss / len(data_loader)

##Defining epoch time function

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

##Instantiating and running the Model

In [ ]:
INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(trg_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 4
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_EPOCHS = 10
CLIP = 1

In [ ]:
subset_loader = DataLoader(dataset, batch_size=32, num_workers=0, sampler=torch.utils.data.SubsetRandomSampler(range(3200)))

In [ ]:
len(subset_loader)

100

In [ ]:
best_valid_loss = float('inf')

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT,  debug=True)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT,  debug=True)

model = Seq2Seq(enc, dec, device,  debug=True).to(device)
model.apply(init_weights)

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)
optimizer = optim.Adam(model.parameters())

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(model, subset_loader, optimizer, criterion, CLIP, debug=True)
    valid_loss = evaluate(model, data_val_loader, criterion, True)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


model.load_state_dict(torch.load('tut1-model.pt'))
test_loss = evaluate(model, data_test_loader, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')



-----------------
Training
-----------------
->num of 32 batches
100


Processing Batches:   1%|          | 1/100 [00:05<08:35,  5.21s/it]


-----------------
Training
-----------------
->batch type
<class 'dict'>

-----------------
Enocder
-----------------
->Embedded
torch.Size([11, 32, 256])
->Outputs
torch.Size([11, 32, 512])

-----------------
Seq2Seq
-----------------
->src seq2seq
torch.Size([11, 32])
->trg seq2seq
torch.Size([11, 32])
->Seq2Sew hidden:
torch.Size([4, 32, 512])
->input to seq2seq
torch.Size([32])

-----------------
Decoder
-----------------
->Input
torch.Size([1, 32])
->Embedded
torch.Size([1, 32, 256])
->Hidden
torch.Size([4, 32, 512])
->Prediction
torch.Size([32, 9167])


Processing Batches:   3%|▎         | 1/39 [00:04<03:02,  4.80s/it]


-----------------
Evaluation
-----------------
->src sha[e]
torch.Size([11, 32])
->trg sha[e]
torch.Size([11, 32])


...

Epoch: 02 | Time: 10m 40s
	Train Loss: 5.503 | Train PPL: 245.318
	 Val. Loss: 5.718 |  Val. PPL: 304.249

-----------------
Training
-----------------
->num of 32 batches
100


Processing Batches:   1%|          | 1/100 [00:04<07:50,  4.75s/it]


-----------------
Training
-----------------
->batch type
<class 'dict'>


Processing Batches:   2%|▏         | 2/100 [00:13<10:56,  6.70s/it]


KeyboardInterrupt: ignored

In [ ]:
def translate_sentence(sentence, src_d,trg_d,trg_d_ind, avg_src, model, device):

    model.eval()

    src_text = [src_d.get(word.lower(), 0) for word in sentence.split()]
    src_text = [0] + src_text + [1]
    src_text = pad_sequence(sequence=src_text, target_size=avg_src)

    src = torch.tensor(src_text).to(device)
    src = src.unsqueeze(1)


    with torch.no_grad():
           hidden, cell = model.encoder(src)

    #first input to the decoder is the <sos> tokens
    trg_indexes = [trg_d["<sos>"]]
    pred_token = ""

    while pred_token != trg_d["<eos>"]:

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        print(trg_indexes[-1])

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        pred_token = output.argmax(1)

        trg_indexes.append(pred_token)


    trg_tokens = [trg_d_ind[i.item()] for i in trg_indexes[1:]]
    print(trg_indexes)
    print(trg_tokens)
    return trg_tokens

In [ ]:
example = 23
src = original_li[31]
trg = modern_li[31]
print(src)
print(trg)

In [ ]:
flip_dict(trg_d)

In [ ]:
pred = translate_sentence(src, src_d, trg_d, flip_dict(trg_d), avg_src, model, device)

#Seq2Seq pretrained transformers model

##Loading the pretrained model

In [ ]:
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

##Dataset for the transformer

In [ ]:
class ShakespeareBartDataset(Dataset):
    def __init__(self, original_sentences, modern_sentences, debug = False):
        self.original_sentences = original_sentences
        self.modern_sentences = modern_sentences
        self.debug = DEBUG

    def __len__(self):
        return len(self.original_sentences)

    def __getitem__(self, idx):
        input_text = self.original_sentences[idx]
        target_text = self.modern_sentences[idx]

        # Tokenize input and target texts
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=11, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors="pt", max_length=11, truncation=True)

        # Pad sentences to the specified max_length
        input_ids = torch.nn.functional.pad(input_ids, (0, 11 - input_ids.shape[-1]))
        target_ids = torch.nn.functional.pad(target_ids, (0, 11 - target_ids.shape[-1]))

        labels = target_ids[:, 1:]

        # Attention masks
        input_attention_mask = (input_ids != 0).float()
        target_attention_mask = (target_ids != 0).float()

        if self.debug:
            # print("Original Sentence:", input_text)
            # print("Tokenized Input IDs:", input_ids)
            # print("Modern Sentence:", target_text)
            # print("Tokenized Target IDs:", target_ids)
            # print("Input Attention Mask:", input_attention_mask)
            # print("Target Attention Mask:", target_attention_mask)
            # print("Labels:", labels)
            pass

        sample = {
            'input_ids': input_ids.squeeze(dim=1),
            'output_sentence': target_text,
            'input_attention_mask': input_attention_mask.squeeze(dim=1),
            'decoder_input_ids': target_ids.squeeze(dim=1),
            'decoder_attention_mask': target_attention_mask.squeeze(dim=1),
            'labels': labels.squeeze(dim=1)
        }

        return sample

##Creating DataLoaders

In [ ]:
dataset = ShakespeareBartDataset(original_li, modern_li)
dataset_val = ShakespeareBartDataset(original_val, modern_val)
dataset_test = ShakespeareBartDataset(original_test, modern_test)

batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
data_val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
data_test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [ ]:
len(data_loader)

527

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


##Training function

In [ ]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for batch in enumerate(tqdm(dataloader, desc="Processing Batches")):
        # print(batch[1]['input_ids'].shape)
        # print(batch[1]['input_attention_mask'].shape)
        # print(batch[1]['decoder_input_ids'].shape)
        # print(batch[1]['decoder_attention_mask'].shape)
        input_ids = batch[1]['input_ids'].squeeze(dim=1).to(device)
        attention_mask = batch[1]['input_attention_mask'].squeeze(dim=1).to(device)
        labels = batch[1]['labels'].squeeze(dim=1).to(device)
        decoder_input_ids = batch[1]['decoder_input_ids'].squeeze(dim=1).to(device)
        decoder_attention_mask = batch[1]['decoder_attention_mask'].squeeze(dim=1).to(device)


        optimizer.zero_grad()
        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    return average_loss

##Evaluation function

In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for batch in enumerate(tqdm(dataloader, desc="Processing Batches")):
            input_ids = batch[1]['input_ids'].squeeze(dim=1).to(device)
            attention_mask = batch[1]['input_attention_mask'].squeeze(dim=1).to(device)
            labels = batch[1]['labels'].squeeze(dim=1).to(device)
            decoder_input_ids = batch[1]['decoder_input_ids'].squeeze(dim=1).to(device)
            decoder_attention_mask = batch[1]['decoder_attention_mask'].squeeze(dim=1).to(device)


            optimizer.zero_grad()
            outputs = model(
                input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss

            total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    return average_loss


In [ ]:
num_epochs = 10

In [ ]:
#best_val_loss = float('inf')


for epoch in range(num_epochs):

    train_loss = train(model, data_loader, optimizer, criterion, device)
    print(f"Epoch {epoch + 1}/{num_epochs} - Training Loss: {train_loss}")


    eval_loss = evaluate(model, data_val_loader, criterion, device)
    print(f"Epoch {epoch + 1}/{num_epochs} - Evaluation Loss: {eval_loss}")


    #if eval_loss < best_val_loss:
       # best_val_loss = eval_loss
       # best_model_state_dict = model.state_dict()

Processing Batches: 100%|██████████| 527/527 [03:35<00:00,  2.45it/s]


Epoch 1/10 - Training Loss: 1.7003621944213954


Processing Batches: 100%|██████████| 66/66 [00:06<00:00,  9.53it/s]


Epoch 1/10 - Evaluation Loss: 1.5517109090631658


Processing Batches: 100%|██████████| 527/527 [03:34<00:00,  2.46it/s]


...

Epoch 3/10 - Training Loss: 0.8912790580537785


Processing Batches: 100%|██████████| 66/66 [00:06<00:00,  9.53it/s]


Epoch 3/10 - Evaluation Loss: 1.6898302056572654


Processing Batches:   2%|▏         | 10/527 [00:04<03:31,  2.45it/s]


KeyboardInterrupt: ignored

In [ ]:
model_name = "/content/saved_weights"
model.save_pretrained(model_name)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

def generate_predictions(model, dataloader, tokenizer, device):
    model.eval()
    predictions = []
    references = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Generating Predictions"):
            input_ids = batch['input_ids'].squeeze(dim=1).to(device)
            attention_mask = batch['input_attention_mask'].squeeze(dim=1).to(device)
            labels = batch['output_sentence']

            # Generate output by decoding the model's predictions
            output_ids = model.generate(input_ids, max_length=50, num_beams=5, length_penalty=0.6, attention_mask=attention_mask)  # Adjust parameters as needed

            # Convert output_ids to tokens
            generated_tokens = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            predictions.extend(generated_tokens)

            # Extend references with labels
            references.extend(labels)

    return predictions, references

# Assuming you have a validation dataloader 'data_val_loader'
generated_predictions,reference_sentences = generate_predictions(model, data_val_loader, tokenizer, device)


Generating Predictions:   0%|          | 0/66 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1298: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
Generating Predictions: 100%|██████████| 66/66 [03:48<00:00,  3.46s/it]


In [ ]:
reference_sentences

["I'm telling you , it's hers .",
 'Good morrow , neighbor , whither away so fast ?',
 "Have them click their heels together and don't dare touch a hair of the master's horse's tail till they kiss their hands .",
 'Saw who ?',
 "I've never been taught how to make anything .",
 "Take this mad wretch to jail. , Father-in-law Baptista , will you see that he's available to appear in court ?",
 "Oh , no , that was my lady's voice .",
 'Did she see you there the whole time , old boy ?',
 "And so the Jew may lawfully claim a pound of flesh nearest the merchant's heart , to be cut off by him. , But please have mercy .",
 "I'm sure he's out there dancing .",
 "I'm glad to see you in such a merry mood .",
 'Do you know the way to Dover ?',
 'Take him to my sleeping area .',
 'My husband .',
 "What's that noise ?",
 "Let's have the evidence .",
 'Goodbye , Helena .',
 "I'll go and join my men .",
 'Here ,',
 'Hunt them down .',
 "What's amazing is how long he lasted .",
 'My soul depends on it , 

In [ ]:
generated_predictions

["I'm sure it is her handwriting is perfect., Oh, it's definitely her handwriting!, It's certainly her handwriting., I don't know what it's written!, What does it say?, What's it say",
 "Good morning, my good morning, neighbor, let's get out of here., Let's go to her house and tell her about your proposal., Come on, tell her my proposal, please, let me tell her, tell",
 'Let them bow with their left legs, as if they were on stony ground. Let them curtsy with their right feet, as though they were about to be boiled boiled for boiled meat!, So please, let them bow.',
 'Who did it see?, Oh, I don\'t know, but I know what it said, It said, "Love me and don\'t give me love.", Love me and tell me, love me, Give me love.',
 'I haven\'t been taught how to make something from nothing! " Well, I\'m not sure I\'m ready to make anything from nothing, but I\'m certainly willing to try! " I don\'t know what you mean, that\'s',
 "Take this crazy rascal to jail and make him pay for his insolence. Take

In [ ]:
bleu_score = sentence_bleu(reference_sentences, generated_predictions)


In [ ]:
print(f"BLEU Score: {bleu_score}")

BLEU Score: 0
